# Building a Simple GIS with Yelp API and Folium - Lab

## Introduction 

So we have learned quite a bit about APIs and how they are now big buzzword in the tech industry. Think of it as a protocol for how to make requests and communicate with another server. We have seen how to mine Twitter for getting text data and apply basic frequency based NLP techniques to get some insight. 

One of the key aspects of being a data scientist is the ability to learn how a new API works, how to go through its specific authentication process (OAuth) and how to process the data structures that get returned as a response to our requests. It is a good practice to spend some time learning the API through the official documentation before sending in requests. 

On these lines, this lab requires you to learn another popular API (YELP Fusion) by following the provided detailed online documentation. We shall build a simple Geographical Information System (GIS) using the data from yelp.

## Objectives
You will be able to: 
* Successfully sign up for Yelp API 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Create a simple geographical system on to view information about selected businesses, at a given location. 


## The Yelp Fusion API - v3


### Point your browser over to this [yelp page](https://www.yelp.com/developers/v3/manage_app) and create an app in order to obtain  `client_id` and `api_key` tokens. 

**NOTE:** You will be required to sign up using Google or Facebook etc. if you dont already have an account.

<img src="yelp_app.png" width=500>



After registration, you'll be presented with your account information and limits of your access. For Yelp, or any other API for that reason, you need to make sure that you dont surpass your request quota, otherwise, you may end up getting banned in some cases. Yelp shows this information to you as below:

<img src="quota.png" width=500>

### Save your api_key and client_id in the variables below:

In [1]:
# Save your tokens in the following string variables
client_id = 'GmJXhjQEMv0k_kMCEksCYw'
api_key = 'fVgQ7j5dVee48eapo1aU9GD2yAeAGgeqE7Xlnhhv3GlwDcaAuma-syhR_z_Qrr-Ar93emGuMy4hiYA1Fija-Ecp6NfYmDOBZadWizN4R8N-Y4sj-sFAP9Kd-lK_iW3Yx'

## The `yelpapi` 

The yelpapi is a pure Python implementation of the Yelp Fusion API (aka Yelp v3 API). It is simple, fast, and robust to any changes Yelp may make to the API in the future. See tha basic usage of this library on the [official Github repo](https://github.com/gfairchild/yelpapi). You may look out for other APIs to achieve this but for this lesson, we shall use it for sake of simplicity. 

First you must pip install the library

### Import `yelpapi ` into working environment and pass in the api_key as shown in the Github Repo 

In [4]:
# Code here
from yelpapi import YelpAPI

yelp_api = YelpAPI(api_key)

### The Api request and response

Great so we can now start making API calls using the format:
```python
response = yelp_api.search_query(term =<search term>, 
                                 location=<search location>, 
                                 sort_by='rating', 
                                 limit=50)
```
We can pass in a lot more arguments to refine our search. [Here is a complete list of options that search API provides us](https://www.yelp.com/developers/documentation/v3/business_search)

* Make an API request using a simple criteria location and term
* save the response as `response` 
* inspect the type and contents of `response`.

In [10]:
# For this example, we are looking for chinese food in London.

term = 'Chinese food'
location = 'London'

In [20]:
# Make an API call using chosen term and location

response = yelp_api.search_query(term=term, location=location, sort_by='rating', limit=10)
type(response)
print(response)


{'businesses': [{'id': 'QE5aa5N-dbfvafLsjn7TVg', 'alias': 'bamboo-flute-fitzrovia', 'name': 'Bamboo Flute', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/kYQZjHmzt8zy1634OsAsxQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/bamboo-flute-fitzrovia?adjust_creative=GmJXhjQEMv0k_kMCEksCYw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GmJXhjQEMv0k_kMCEksCYw', 'review_count': 29, 'categories': [{'alias': 'chinese', 'title': 'Chinese'}], 'rating': 4.5, 'coordinates': {'latitude': 51.5228746, 'longitude': -0.1418939}, 'transactions': [], 'price': '£', 'location': {'address1': '145 Cleveland Street', 'address2': '', 'address3': '', 'city': 'Fitzrovia', 'zip_code': 'W1T 6QH', 'country': 'GB', 'state': 'XGL', 'display_address': ['145 Cleveland Street', 'Fitzrovia W1T 6QH', 'United Kingdom']}, 'phone': '+442073872738', 'display_phone': '+44 20 7387 2738', 'distance': 1189.5082547156321}, {'id': 'h2cCoDNQOPd51HFwvuAGNg', 'alias': 'hakkasan-london-3', 'n

### JSON .. again ! 

We have a nice nifty little return now! As you can see, the contents of the response is formatted as a string but what kind of data structures does this remind you of?  

To start there's the outer curly brackets:  

#### {"businesses":   

Hopefully you're thinking 'hey that's just like a python dictionary!'

Then within that we have what appears to be a list of dictionaries:  

#### {"id": "jeWIYbgBho9vBDhc5S1xvg",

This response is an example of a JSON (JavaScript Object Notation) format that we've seen so many times before. We can simply treat it as a dictionary and process it further. 

### Inspect the values for all the keys in the response

In [21]:
# inspect the key value pairs to understand the strcuture of data 

for key in response.keys():
    value = response.values()
    print('Key: {}, Value: {}'.format(key, type(value)))


Key: businesses, Value: <class 'dict_values'>
Key: total, Value: <class 'dict_values'>
Key: region, Value: <class 'dict_values'>


Whoops, what's going on here!? Well, notice from our previous preview of the response that we saw there were a hierarhcy within the response. Let's begin to investigate further to see what the problem is.

First, recall that the overall strucutre of the response was a dictionary. Let's look at what the keys are:

In [22]:
response.keys()

dict_keys(['businesses', 'total', 'region'])

Consult the Yelp API and learn what value is carried in each key. 

#### Continue to preview these keys  further to get a little better acquainted. 

In [23]:
print('BUSINESS:', response['businesses'][0])

print('REGION:', response['region'])

print('TOTAL :',response['total'])

BUSINESS: {'id': 'QE5aa5N-dbfvafLsjn7TVg', 'alias': 'bamboo-flute-fitzrovia', 'name': 'Bamboo Flute', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/kYQZjHmzt8zy1634OsAsxQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/bamboo-flute-fitzrovia?adjust_creative=GmJXhjQEMv0k_kMCEksCYw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GmJXhjQEMv0k_kMCEksCYw', 'review_count': 29, 'categories': [{'alias': 'chinese', 'title': 'Chinese'}], 'rating': 4.5, 'coordinates': {'latitude': 51.5228746, 'longitude': -0.1418939}, 'transactions': [], 'price': '£', 'location': {'address1': '145 Cleveland Street', 'address2': '', 'address3': '', 'city': 'Fitzrovia', 'zip_code': 'W1T 6QH', 'country': 'GB', 'state': 'XGL', 'display_address': ['145 Cleveland Street', 'Fitzrovia W1T 6QH', 'United Kingdom']}, 'phone': '+442073872738', 'display_phone': '+44 20 7387 2738', 'distance': 1189.5082547156321}
REGION: {'center': {'longitude': -0.135955810546875, 'latitude': 51.5128

This makes more sense, so we are mainly interested in the `businesses` for our needs. 

### Print the names of businesses and included ratings 

In [24]:
# Code here 
for i in range(10):
    print(response['businesses'][i]['name'])

Bamboo Flute
Hakkasan
Yauatcha
Lanzhou Noodle Bar
Hakkasan Mayfair
Silk Road
Hunan
Bugis Street Brasserie
Barshu
Gold Mine


Great, now are are getting somewhere. It is a good idea at this stage to store this information as a dataframe for processing further. 
### Create a Pandas dataframe for contents of `businesses`
* Check the number of records in the dataframe
* Inspect the columns and head

In [25]:
# Code here 
import pandas as pd

df = pd.DataFrame.from_dict(response['businesses'])

print(len(df))
print(df.columns)
df.head()

10
Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,bamboo-flute-fitzrovia,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'latitude': 51.5228746, 'longitude': -0.1418939}",+44 20 7387 2738,1189.508255,QE5aa5N-dbfvafLsjn7TVg,https://s3-media3.fl.yelpcdn.com/bphoto/kYQZjH...,False,"{'address1': '145 Cleveland Street', 'address2...",Bamboo Flute,+442073872738,£,4.5,29,[],https://www.yelp.com/biz/bamboo-flute-fitzrovi...
1,hakkasan-london-3,"[{'alias': 'cantonese', 'title': 'Cantonese'}]","{'latitude': 51.5171482803943, 'longitude': -0...",+44 20 7927 7000,558.965004,h2cCoDNQOPd51HFwvuAGNg,https://s3-media2.fl.yelpcdn.com/bphoto/IcS9yz...,False,"{'address1': '8 Hanway Place', 'address2': '',...",Hakkasan,+442079277000,££££,4.0,216,[],https://www.yelp.com/biz/hakkasan-london-3?adj...
2,yauatcha-london-7,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...","{'latitude': 51.5137076071076, 'longitude': -0...",+44 20 7494 8888,112.598410,sYwBQ7mJYhB35nn-_SZstQ,https://s3-media1.fl.yelpcdn.com/bphoto/G7Ydt9...,False,"{'address1': '15-17 Broadwick Street', 'addres...",Yauatcha,+442074948888,£££,4.0,456,[],https://www.yelp.com/biz/yauatcha-london-7?adj...
3,lanzhou-noodle-bar-london,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...","{'latitude': 51.5116034713013, 'longitude': -0...",+44 20 7467 4546,578.474355,pdFiFtol9YI__9ROOXUIYA,https://s3-media2.fl.yelpcdn.com/bphoto/L5PMp7...,False,"{'address1': '33 Cranbourne Street', 'address2...",Lanzhou Noodle Bar,+442074674546,£,4.0,301,[],https://www.yelp.com/biz/lanzhou-noodle-bar-lo...
4,hakkasan-mayfair-london-2,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'latitude': 51.5103202323384, 'longitude': -0...",+44 20 7907 1888,681.445072,chEEcQbc8PbidTeXK34H9g,https://s3-media2.fl.yelpcdn.com/bphoto/LX_W20...,False,"{'address1': '17 Bruton Street', 'address2': '...",Hakkasan Mayfair,+442079071888,££££,4.0,116,[],https://www.yelp.com/biz/hakkasan-mayfair-lond...


This is fantastic. We have successfully learned a new API , made requests to it, recieved and studied the response and stored the results in a dataframe and can now enjoy all the goodness of Pandas. Thats quite a bit of data engineering. 

### Visualize the location from search query
The `region` key in the response carries the geographical information for the region searched.
* Get the latitude / longitude information from `region`
* Create a folium map with these coordinates. 
* Use a zoom start value = 13

In [26]:
import folium

# Code here

#Retrieve the Latitude and Longitude from the Yelp Response
lat_long = response['region']['center']
lat = lat_long['latitude']
long = lat_long['longitude']

#Create a map of the area
yelp_map = folium.Map([lat, long], zoom_start=13)
yelp_map


Expected Output:
![](london.png)

Nice. We can now extract the coordinate information for each business and plot it on this map.

### Get the business coordinates from dataframe for each business and plot on the map above

In [27]:
# Code here 

for row in df.index:
    
    lat_long = df['coordinates'][row]
    lat = lat_long['latitude']
    long = lat_long['longitude']

    marker = folium.Marker(location=[lat,long])
    marker.add_to(yelp_map)
yelp_map

Expected output:
![](markers.png)

Kool so we have everything in place but the visualization is still not very *Informative* so to speak. You can't tell which marker represents which business and also other information on business like rating, cost, links to user reviews etc. is still not visible. SO its geographical , but not exactly an Information System yet as you cant make any decisions on this information.  Here's as example of what it possible can look like
![](out.png)


For this you need to understand `folium.popup()` which let's you click on a marker to show a pop up window. This window acts more like an HTML page so you can easily format the information you present in the popup using following values:
* The official business logo/image:  `image_url`
* Name of the Business: `name`
* Price (how expensive): `price`
* Links to user reviews on Yelp: `url`

Doing this in HTML is not required , so we recommend that you first try to put in basic information in the popups as just text. As a next stage , you can start changing into HTML code to make it visually more appealing.

### Attempt to recreate the interactive visualization shown above.

Here's a good resource with code examples on [how to create folium popups](https://github.com/python-visualization/folium/blob/master/examples/Popups.ipynb)

In [28]:
# Code here 
from folium import IFrame

for row in df.index:
   
    lat_long = df['coordinates'][row]
    lat = lat_long['latitude']
    long = lat_long['longitude']

    name = 'Business Name: ' + str(df['name'][row])
    rating = 'Rating: ' + str(df['rating'][row])
    price = 'Price: ' + str(df['price'][row])

    img = df['image_url'][row]
    txt = folium.Html('<head><style> body {background-image: url('+ img+ ');\
                        background-color: rgba(255,255,255,0.6);background-blend-mode: lighten;\
                        background-position: center; background-repeat: no-repeat; background-size: cover;}</style></head>' +
                      '<H2><center><font color="black">' + str(df['name'][row]) + '</H2></center>' +\
                      '<B><center>Rating: </B>' + str(df['rating'][row]) + '/ 5</br></center>' +\
                      '<B><center>Price: </B>' + str(df['price'][row]) + '</br></center>' +\
                      '<center><a href='+ str(df['url'][row])+ ' target="_blank">Read Reviews</b></a></br></br></center>' + \
                      '<small>Searching for '+ term + ' in ' + location, script=True)

    iframe = folium.IFrame(txt, width=400, height=150)
    popup = folium.Popup(iframe, max_width=3000)
    Text = folium.Marker(location=[lat,long], popup=popup,icon=folium.Icon(icon_color='green'))
    yelp_map.add_child(Text)

yelp_map


Wow . An Interactive Geographical information System backed by live data through API calling. 

<img src="star.jpg" width=300>

### More APIs to Checkout

* Google Maps
* Twitter
* AWS
* IBM's Watson
* Yelp

## Summary 

In this lab, we learned how to use the Yelp API with authentication, making calls, understanding the responses and creating interactive geographical visualizations in Folium. We encourage you to re-visit this lab again once you have studied some important machine learning algorithms to make predictions , find similarities, group/cluster businesses or classify them based on user criteria. 